In [1]:
import pandas as pd
import spacy
import os
import scispacy
from spacy import displacy
from tqdm.notebook import tqdm

In [2]:
vec_df = pd.concat([pd.read_pickle("/home/acorn/Documents/covid/v6/v6_text/" + f, compression="gzip") for f in os.listdir("/home/acorn/Documents/covid/v6/v6_text/")])
#vec_df = pd.concat([vec_df, pd.read_csv("/home/acorn/Documents/covid/v7/v7_text.json", orient="records")])

In [ ]:
sentences = df[df["sentence"].str.contains("population") | (df["sentence"].str.contains("sample") & df["sentence"].str.contains("size")]["sentence"]
sentences.reset_index(drop=True,inplace=True)
nlp = spacy.load("en_core_sci_lg")
doc = nlp(sentences[0])
displacy.serve(doc, style="dep")

In [ ]:
subject = doc[7]

In [ ]:
subject = doc[7]
a = [i.dep_ for i in subject.lefts]
b = [i for i in subject.lefts]
c = [i for i in doc[2].ancestors]

In [ ]:
vec_df.fillna("[]",inplace=True)
npr = list(set(vec_df[vec_df["sentence"].str.contains("not peer")]["paper_id"].to_list()))

In [ ]:
len(npr)

In [2]:
import pandas as pd
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import os

In [3]:
def bulk_index(df, index_name): 
    # This method will process rows of a given dataframe
    # into dicts indexable by ES. Those are then indexed.
    
    es = Elasticsearch()

    helpers.bulk(es, doc_generator(df, index_name))
    
def doc_generator(df, index_name):
    # This function creates documents from dataframe rows, and then 
    # indexes those documents in ElasticSearch
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": index_name,
                "article": document.to_dict(),
            }
    return True

In [14]:
directory = "/home/acorn/Documents/covid/v6/v6_text/"
vec_df = pd.concat([pd.read_pickle(directory + f, compression="gzip") for f in os.listdir(directory)])

In [15]:
v7 = pd.read_json("/home/acorn/Documents/covid/v7/v7_text.json", orient="records")
if "sentenc_id" in v7.columns:
    v7.rename(columns={"sentenc_id": "sentence_id"}, inplace=True)

In [16]:
df = pd.concat([vec_df, v7])
del v7
del vec_df

In [4]:
df = pd.read_pickle("v6-7bySection.pkl", compression="gzip")
bulk_index(df, index_name="cordv7sections")
del df 
print("Sections done.")
df = pd.read_pickle("v6-7bySentence.pkl",compression="gzip")
bulk_index(df, index_name="cordv7sentences")
del df
print("Sentences done")

KeyboardInterrupt: 

In [17]:
df.fillna("[]", inplace=True)
column_remap = {'lemma':'sum', 
                'UMLS':'sum', 
                'GGP':'sum', 
                'SO':'sum', 
                'TAXON':'sum', 
                'CHEBI':'sum', 
                'GO':'sum', 
                'CL':'sum', 
                'DNA':'sum',
                'CELL_TYPE':'sum', 
                'CELL_LINE':'sum', 
                'RNA':'sum', 
                'PROTEIN':'sum', 
                'DISEASE':'sum', 
                'CHEMICAL':'sum', 
                'CANCER':'sum', 
                'ORGAN':'sum', 
                'TISSUE':'sum', 
                'ORGANISM':'sum', 
                'CELL':'sum', 
                'AMINO_ACID':'sum', 
                'GENE_OR_GENE_PRODUCT':'sum', 
                'SIMPLE_CHEMICAL':'sum', 
                'ANATOMICAL_SYSTEM':'sum',
                'IMMATERIAL_ANATOMICAL_ENTITY':'sum', 
                'MULTI-TISSUE_STRUCTURE':'sum', 
                'DEVELOPING_ANATOMICAL_STRUCTURE':'sum',
                'ORGANISM_SUBDIVISION':'sum', 
                'CELLULAR_COMPONENT':'sum', 
                'PATHOLOGICAL_FORMATION':'sum', 
                'ORGANISM_SUBSTANCE':'sum'}
for col in tqdm(column_remap.keys()):
    df[col] = [i.split("~") if isinstance(i, str) else i for i in df[col].to_list()]
df.to_pickle("v6-7bySentence.pkl",compression="gzip")

In [11]:
df.fillna("[]", inplace=True)
sentences = df.groupby(['paper_id','section'])['sentence'].apply(' '.join).reset_index()["sentence"].to_list()
print("Made sentences")
df = df.groupby(['paper_id','section']).agg(column_remap)
print("Grouped and aggregated.")
df["sentence"] = sentences
print("Assigned column")
df.to_pickle("v6-7bySection.pkl", compression="gzip")

Made sentences
Grouped and aggregated.
Assigned column


In [13]:
del df

NameError: name 'df' is not defined